In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

class SimpleANN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        # Initialize weights and biases
        self.learning_rate = learning_rate
        self.weights_input_hidden = np.random.rand(input_size, hidden_size) - 0.5
        self.weights_hidden_output = np.random.rand(hidden_size, output_size) - 0.5
        self.bias_hidden = np.random.rand(1, hidden_size) - 0.5
        self.bias_output = np.random.rand(1, output_size) - 0.5

    def forward(self, X):
        self.hidden_layer_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_output = sigmoid(self.hidden_layer_input)
        
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_hidden_output) + self.bias_output
        self.output = sigmoid(self.output_layer_input)
        return self.output

    def backward(self, X, y):
        output_error = y - self.output
        output_delta = output_error * sigmoid_derivative(self.output)
        
        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden_layer_output)

        self.weights_hidden_output += self.hidden_layer_output.T.dot(output_delta) * self.learning_rate
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * self.learning_rate
        self.weights_input_hidden += X.T.dot(hidden_delta) * self.learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0, keepdims=True) * self.learning_rate

    def train(self, X, y, epochs=1000):
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y)
            
            if epoch % 100 == 0:
                loss = mean_squared_error(y, self.output)
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        return self.forward(X)

df = pd.read_csv('binary_soil_measures.csv')

X = df[['N', 'P', 'K', 'ph']].values
y = df['crop'].values

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = y.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

input_size = X_train.shape[1]
hidden_size = 16
output_size = 1
learning_rate = 0.01
ann = SimpleANN(input_size, hidden_size, output_size, learning_rate)
ann.train(X_train, y_train, epochs=1000)

y_pred = ann.predict(X_test)
y_pred_class = (y_pred > 0.5).astype(int)

accuracy = np.mean(y_pred_class == y_test)
print(f"Test Accuracy: {accuracy:.4f}")




Epoch 0, Loss: 136.86443689811637
Epoch 100, Loss: 132.7935064935065
Epoch 200, Loss: 132.7935064935065
Epoch 300, Loss: 132.7935064935065
Epoch 400, Loss: 132.7935064935065
Epoch 500, Loss: 132.7935064935065
Epoch 600, Loss: 132.7935064935065
Epoch 700, Loss: 132.7935064935065
Epoch 800, Loss: 132.7935064935065
Epoch 900, Loss: 132.7935064935065
Test Accuracy: 0.0424
